In [75]:
import pdfplumber
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

def skaityti_pdf(kelias_iki_pdf, kalba='lt'):
    tekstas = []
    zodziu_kiekis = 0
    with pdfplumber.open(kelias_iki_pdf) as pdf:
        for puslapis in pdf.pages:
            puslapio_tekstas = puslapis.extract_text()
            if kalba != 'lt':
                if zodziu_kiekis < 50000:
                    if puslapio_tekstas:
                        tekstas.append(puslapio_tekstas)
                        if isinstance(puslapio_tekstas, str):
                            zodziu_kiekis += len(puslapio_tekstas.split())
                            # print(zodziu_kiekis)
                else:
                    break
            else:
                if puslapio_tekstas:
                    tekstas.append(puslapio_tekstas)
    return tekstas

def tvarkyti_teksta(tekstas):
    sutvarkyti_puslapiai = [puslapis.replace("-\n", "").replace("\n", " ") for puslapis in tekstas]
    return sutvarkyti_puslapiai

# sukurti funkcija, kuri paverstu sakiniu sarasus i zodziu sarasus
def skaidyti_i_zodzius(tekstas):
    zodziai = []
    for puslapis in tekstas:
        zodziai.extend(puslapis.split())
    return zodziai

# Skaitymas ir apdorojimas PDF failu
lietuviskas_tekstas = skaityti_pdf('baltaragio_malunas.pdf')
lietuviski_puslapiai = lietuviskas_tekstas[3:]

angliskas_tekstas = skaityti_pdf('SherlockHolmesComplete.pdf', kalba='en')
angliski_puslapiai = angliskas_tekstas

# Sutvarkyti tekstai
sutvarkyti_lt_puslapiai = tvarkyti_teksta(lietuviski_puslapiai)
sutvarkyti_en_puslapiai = tvarkyti_teksta(angliski_puslapiai)


# Sukurti zodziu sarasus
lt_zodziai = skaidyti_i_zodzius(sutvarkyti_lt_puslapiai)
en_zodziai = skaidyti_i_zodzius(sutvarkyti_en_puslapiai)

# kuriant labels, naudoti nauja kintamaji
lt_labels = ["lt" for puslapis in sutvarkyti_lt_puslapiai]
en_labels = ['en' for puslapis in sutvarkyti_en_puslapiai]

# Sukurti bendra tekstu ir labels sarasa
lt_en_puslapiai = sutvarkyti_lt_puslapiai + sutvarkyti_en_puslapiai
lt_en_labels = lt_labels + en_labels

# Mokymas ir testavimas 
X_train, X_test, y_train, y_test = train_test_split(lt_en_puslapiai, lt_en_labels, random_state=42)

# Modelio kurymas ir mokymas
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

# Vertinimas
y_pred = model.predict(X_test)
print(accuracy_score(y_pred, y_test))


0.9711538461538461


In [89]:
print(sutvarkyti_en_puslapiai)
print(sutvarkyti_lt_puslapiai)

['The Complete Sherlock Holmes Arthur Conan Doyle', 'Table of contents AStudyInScarlet.................................................... 1 TheSignoftheFour.................................................. 63 TheAdventuresofSherlockHolmes AScandalinBohemia................................................ 119 TheRed-HeadedLeague............................................. 135 ACaseofIdentity.................................................... 149 TheBoscombeValleyMystery........................................ 159 TheFiveOrangePips................................................. 173 TheManwiththeTwistedLip........................................ 185 TheAdventureoftheBlueCarbuncle................................. 199 TheAdventureoftheSpeckledBand................................. 211 TheAdventureoftheEngineer’sThumb............................. 225 TheAdventureoftheNobleBachelor................................. 237 TheAdventureoftheBerylCoronet.................................. 249 The

In [84]:
words_count = 0
for puslapis in angliskas_tekstas:
    words_count += len(puslapis.split())
print(words_count)

50012


In [86]:
X_train, X_test, y_train, y_test = train_test_split(lt_en_puslapiai, lt_en_labels)

model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(accuracy_score(y_pred, y_test))

0.9711538461538461


In [87]:
def atspek_kalba(tekstas):
    return model.predict([tekstas])

print(atspek_kalba('ačiū'))
print(atspek_kalba('thanks'))
print(atspek_kalba('hello, how are you?'))
print(atspek_kalba('a little dog'))

['lt']
['lt']
['en']
['lt']


In [88]:
test_words = [
    "katė", "namas", "mokykla", "draugas", "mašina",  # LT
    "dog", "building", "university", "companion", "vehicle",  # EN
    "apple", "computer", "book", "river", "happiness",  # EN
    "obuolys", "miegas", "vaikas", "žaidimas", "medis",  # LT
    "garden", "flower", "music", "language", "science",  # EN
    "šeima", "kelionė", "darbas", "maistas", "gimtadienis",  # LT
    "technology", "education", "history", "literature", "art",  # EN
    "lietus", "sniegas", "saulė", "vėjas", "širdis",  # LT
    "mountain", "ocean", "desert", "forest", "beach",  # EN
    "oras", "kambarys", "miestas", "kaimas", "šalis"  # LT
]


for word in test_words:
    predicted_label = atspek_kalba(word)
    print(f"The predicted label for '{word}' is: {predicted_label}")

The predicted label for 'katė' is: ['lt']
The predicted label for 'namas' is: ['lt']
The predicted label for 'mokykla' is: ['lt']
The predicted label for 'draugas' is: ['lt']
The predicted label for 'mašina' is: ['lt']
The predicted label for 'dog' is: ['lt']
The predicted label for 'building' is: ['lt']
The predicted label for 'university' is: ['lt']
The predicted label for 'companion' is: ['lt']
The predicted label for 'vehicle' is: ['lt']
The predicted label for 'apple' is: ['lt']
The predicted label for 'computer' is: ['lt']
The predicted label for 'book' is: ['lt']
The predicted label for 'river' is: ['lt']
The predicted label for 'happiness' is: ['lt']
The predicted label for 'obuolys' is: ['lt']
The predicted label for 'miegas' is: ['lt']
The predicted label for 'vaikas' is: ['lt']
The predicted label for 'žaidimas' is: ['lt']
The predicted label for 'medis' is: ['lt']
The predicted label for 'garden' is: ['lt']
The predicted label for 'flower' is: ['lt']
The predicted label for